In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [2]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
bureau_balance_data = pd.read_csv('dataFiles/bureau_balance.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')
pos_cash_balance_data = pd.read_csv('dataFiles/POS_CASH_balance.csv')
installments_data = pd.read_csv('dataFiles/installments_payments.csv')
cc_data = pd.read_csv('dataFiles/credit_card_balance.csv')

In [3]:
cc_data_one_hot = pd.get_dummies(cc_data, drop_first = True).drop('SK_ID_CURR', axis = 1)

In [4]:
cc_data_grouped = cc_data_one_hot.groupby('SK_ID_PREV', as_index = False).agg(['count', sum, 'median', 'mean', min, max])

In [5]:
cc_data_grouped.columns = ['_CC_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in cc_data_grouped.columns]

In [6]:
filtered_cols = filter(lambda x: x[-5:] != 'count', cc_data_grouped.columns.tolist())
filtered_cols.insert(0, 'MONTHS_BALANCE_CC_count')

In [35]:
cc_data_grouped = cc_data_grouped[filtered_cols].reset_index()

In [28]:
prev_app_data_dummies = pd.get_dummies(prev_app_data[['NAME_CONTRACT_TYPE_PRE','FLAG_LAST_APPL_PER_CONTRACT_PRE','NAME_CONTRACT_STATUS_PRE',\
                                       'NAME_PAYMENT_TYPE_PRE','CODE_REJECT_REASON_PRE','NAME_TYPE_SUITE_PRE','NAME_CLIENT_TYPE_PRE',\
                                       'NAME_PORTFOLIO_PRE','CHANNEL_TYPE_PRE','NAME_YIELD_GROUP_PRE','PRODUCT_COMBINATION_PRE']]).fillna(0)

In [29]:
prev_app_data = pd.concat([prev_app_data.select_dtypes(exclude=['object']), prev_app_data_dummies], axis = 1)

In [30]:
prev_app_data.columns = [i + '_PRE' if i != 'SK_ID_PREV' and i != 'SK_ID_CURR' else i for i in prev_app_data.columns]

In [37]:
prev_app_data_merged = prev_app_data.merge(cc_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [38]:
installments_data_grouped = installments_data.groupby('SK_ID_PREV', as_index = False).agg({'NUM_INSTALMENT_VERSION':[sum, 'median', 'mean', min, max],
                                                              'NUM_INSTALMENT_NUMBER':[sum, 'median', 'mean', min, max],
                                                              'DAYS_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'DAYS_ENTRY_PAYMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_PAYMENT': [sum, 'median', 'mean', min, max]})

In [39]:
installments_data_grouped.columns = ['_INST_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in installments_data_grouped.columns]

In [40]:
prev_app_data_merged = prev_app_data_merged.merge(installments_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [41]:
pos_cash_balance_data = pd.get_dummies(pos_cash_balance_data, drop_first = True)

In [42]:
pos_cash_balance_data_grouped = pos_cash_balance_data.groupby('SK_ID_PREV', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD_DEF':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Amortized debt':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Approved':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Completed':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Returned to the store':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Signed':[sum, 'median', 'mean', min, max],
                                                                                                })

In [43]:
pos_cash_balance_data_grouped.columns = ['_POS_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in pos_cash_balance_data_grouped.columns]

In [44]:
prev_app_data_merged = prev_app_data_merged.merge(pos_cash_balance_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [45]:
bureau_balance_data_grouped = pd.get_dummies(bureau_balance_data).groupby('SK_ID_BUREAU', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_0':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_1':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_2':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_4':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_5':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_X':[sum, 'median', 'mean', min, max]})

In [46]:
bureau_balance_data_grouped.columns = ['_BBD_'.join(col) if col != ('SK_ID_BUREAU', '') else col[0] for col in bureau_balance_data_grouped.columns]

In [47]:
bureau_data_dummies = pd.get_dummies(bureau_data[['CREDIT_ACTIVE','CREDIT_CURRENCY', 'CREDIT_TYPE']]).fillna(0)

In [48]:
bureau_data = pd.concat([bureau_data.select_dtypes(exclude=['object']), bureau_data_dummies], axis = 1)

In [49]:
bureau_data = bureau_data.merge(bureau_balance_data_grouped, how = 'left', left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')

In [50]:
# Joining Bureau Grouped Data with Train Data
bureau_data_grouped = bureau_data.groupby('SK_ID_CURR', as_index=False).agg({'DAYS_CREDIT':[sum, 'median', 'mean', min, max],
                                                                             'CREDIT_DAY_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_ENDDATE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_ENDDATE_FACT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_MAX_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'CNT_CREDIT_PROLONG':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_DEBT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_LIMIT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_UPDATE':[sum, 'median', 'mean', min, max],
                                                                             'AMT_ANNUITY':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_max':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_max':[sum, 'median', 'mean', min, max]})

In [51]:
bureau_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in bureau_data_grouped.columns]

In [52]:
train_dummies = pd.get_dummies(train[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']]).fillna(0)

In [53]:
test_dummies = pd.get_dummies(test[['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',\
'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FONDKAPREMONT_MODE','HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']]).fillna(0)

In [54]:
train_one_hot = pd.concat([train.select_dtypes(exclude=['object']), train_dummies], axis = 1)

test_one_hot = pd.concat([test.select_dtypes(exclude=['object']), test_dummies], axis = 1)

In [55]:
train_merged = train_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_one_hot.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [56]:
prev_app_data_subset = prev_app_data_merged[prev_app_data_merged.select_dtypes(exclude=['object']).columns.tolist()].drop('SK_ID_PREV', axis = 1)

In [57]:
prev_app_data_grouped = prev_app_data_subset.groupby('SK_ID_CURR', as_index=False).agg(sum)

In [58]:
#prev_app_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in prev_app_data_grouped.columns]
#prev_app_data_grouped = prev_app_data_grouped.reset_index()

In [59]:
train_merged = train_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [60]:
own_car_median = train_merged.OWN_CAR_AGE.median()

In [61]:
def own_car_missing(x):
    if x['FLAG_OWN_CAR'] == 'N':
        return 28
    elif x['FLAG_OWN_CAR'] == 'Y' and pd.isnull(x['OWN_CAR_AGE']):
        return own_car_median
    else:
        return x['OWN_CAR_AGE']

In [62]:
train_merged_subset = train_merged.dropna(thresh=len(train_merged) - 100000, axis = 1)

In [63]:
new_cols = train_merged_subset.columns.tolist()
new_cols.remove('TARGET')

In [64]:
len(train_merged_subset.columns)

509

In [65]:
col_corr = train_merged_subset.corr()['TARGET'].sort_values()

In [66]:
column_corr_subset = col_corr[(col_corr >= 0) | (col_corr < 0)].index.values.tolist()

In [67]:
column_corr_subset.remove('TARGET')
column_corr_subset.remove('CODE_GENDER_XNA')
column_corr_subset.remove('NAME_FAMILY_STATUS_Unknown')
column_corr_subset.remove('NAME_INCOME_TYPE_Maternity leave')

In [68]:
train_subset = train_merged_subset[column_corr_subset]

test_subset = test_merged[column_corr_subset]

In [69]:
imputer = Imputer(strategy = 'median')
imputer.fit(train_subset)
train_merged_imputed = pd.DataFrame(imputer.transform(train_subset), columns = train_subset.columns)
test_merged_imputed = pd.DataFrame(imputer.transform(test_subset), columns = train_subset.columns)

In [70]:
train_merged_imputed['DAYS_EMPLOYED_^2'] = train_merged_imputed['DAYS_EMPLOYED'] ** 2
train_merged_imputed['AMT_GOODS_PRICE_^2'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 2
train_merged_imputed['DAYS_CREDIT_^2'] = train_merged_imputed['DAYS_CREDIT_mean'] ** 2
train_merged_imputed['DAYS_BIRTH_^2'] = train_merged_imputed['DAYS_BIRTH'] ** 2

test_merged_imputed['DAYS_EMPLOYED_^2'] = test_merged_imputed['DAYS_EMPLOYED'] ** 2
test_merged_imputed['AMT_GOODS_PRICE_^2'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 2
test_merged_imputed['DAYS_CREDIT_^2'] = test_merged_imputed['DAYS_CREDIT_mean'] ** 2
test_merged_imputed['DAYS_BIRTH_^2'] = test_merged_imputed['DAYS_BIRTH'] ** 2

/Users/MikeTodisco/anaconda2/lib/python2.7/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [71]:
poly_transformer = PolynomialFeatures(degree = 1)
poly_transformer.fit(train_merged_imputed)
train_poly_features = poly_transformer.transform(train_merged_imputed)

In [72]:
train_subset_poly = pd.DataFrame(train_poly_features, columns = poly_transformer.get_feature_names(
    input_features = train_merged_imputed.columns.tolist()
))

In [73]:
test_poly_features = poly_transformer.transform(test_merged_imputed)

In [74]:
test_subset_poly = pd.DataFrame(test_poly_features, columns = poly_transformer.get_feature_names(input_features = test_merged_imputed.columns.tolist()))

In [75]:
scaler = MinMaxScaler(feature_range = (0, 1))

In [76]:
scaler.fit(train_subset_poly)
train_scaled = scaler.transform(train_subset_poly)
test_scaled = scaler.transform(test_subset_poly)

In [77]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.1)

In [78]:
cat_model.fit(train_scaled, train.TARGET)

0:	learn: 0.5857104	total: 1.47s	remaining: 24m 29s
1:	learn: 0.5061143	total: 3.01s	remaining: 25m 3s
2:	learn: 0.4474903	total: 4.46s	remaining: 24m 42s
3:	learn: 0.4027261	total: 6.15s	remaining: 25m 32s
4:	learn: 0.3683854	total: 7.66s	remaining: 25m 23s
5:	learn: 0.3424938	total: 9.09s	remaining: 25m 6s
6:	learn: 0.3234566	total: 10.5s	remaining: 24m 55s
7:	learn: 0.3088042	total: 12.2s	remaining: 25m 13s
8:	learn: 0.2966674	total: 13.8s	remaining: 25m 23s
9:	learn: 0.2870682	total: 15.3s	remaining: 25m 12s
10:	learn: 0.2797087	total: 16.8s	remaining: 25m 6s
11:	learn: 0.2741096	total: 18.5s	remaining: 25m 22s
12:	learn: 0.2697527	total: 20s	remaining: 25m 18s
13:	learn: 0.2662221	total: 21.5s	remaining: 25m 11s
14:	learn: 0.2632408	total: 22.9s	remaining: 25m 4s
15:	learn: 0.2606547	total: 24.3s	remaining: 24m 55s
16:	learn: 0.2586181	total: 25.7s	remaining: 24m 48s
17:	learn: 0.2571308	total: 27.2s	remaining: 24m 45s
18:	learn: 0.2559329	total: 28.6s	remaining: 24m 37s
19:	learn

In [79]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']

In [80]:
#max_sub_cat = submission_cat.Target.max()
#min_sub_cat = submission_cat.Target.min()

In [81]:
#submission_cat['Target'] = submission_cat['Target'].apply(lambda x: (x - min_sub_cat) / (max_sub_cat - min_sub_cat))

In [82]:
submission_cat.to_csv('reduced_cat19.csv', index = False)